# ELYZA-tasks-100に対する応答の生成

本サンプルは`Google Colab`(GPU-T4) 環境での実行を想定して作成しております。

#### 作成するjsonlファイル
ELYZA-tasks-100の`input`に対する応答を、Hugging Faceに公開してあるLLMモデル（"llm-jp/llm-jp-3-3.7b"など）を使用して生成します。


In [1]:
!nvidia-smi

Sat Nov 30 02:13:00 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8              13W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### 1.モジュールの導入

In [2]:
!pip install -U bitsandbytes
!pip install -U transformers
!pip install -U accelerate
!pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 108.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.2
    Uninstalling transformers-4.46.2:
      Successfully uninstalled transformers-4.46.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's

### 2.HFのトークン設定

必要な場合はHugging Faceトークンを設定してください

In [6]:
# Hugging Face Token を指定
# 下記の URL から Hugging Face Token を取得できますので下記の HF_TOKEN に入れてください。
# https://huggingface.co/settings/tokens
HF_TOKEN = "" #@param {type:"string"}

# あるいは Google Colab シークレットを使う場合、左のサイドバーより🔑マークをクリック
# HF_TOKEN という名前で Value に Hugging Face Token を入れてください。
# ノートブックからのアクセスのトグルをオンにし、下記の二行のコードのコメントアウトを外してください。

# from google.colab import userdata
# HF_TOKEN=userdata.get('HF_TOKEN')

### 3.Githubリポジトリからツールのダウンロード

In [3]:
#リポジトリコード
!git clone  https://github.com/tana114/elyza-tasks-grader-like
#ディレクトリの移動
%cd /content/elyza-tasks-grader-like

Cloning into 'elyza-tasks-grader-like'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 22 (delta 2), reused 19 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (22/22), 17.97 KiB | 17.97 MiB/s, done.
Resolving deltas: 100% (2/2), done.
/content/elyza-tasks-grader-like


In [4]:
#ディレクトリの移動
%cd /content/elyza-tasks-grader-like
!ls

/content/elyza-tasks-grader-like
client	data  elyza_task  model  README.md  util


### 4.ツールを使用した応答の生成

#### 使用するモデルの準備

In [7]:
from logging import DEBUG, INFO, WARN, ERROR, basicConfig
basicConfig(level=WARN)

import torch
from transformers import PreTrainedModel, PreTrainedTokenizer
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

load_in_4bit = True
model_name = "llm-jp/llm-jp-3-3.7b"


# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=load_in_4bit,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False,
)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    token = HF_TOKEN
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True,
    token = HF_TOKEN)



config.json:   0%|          | 0.00/683 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/21.0k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.57G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/494 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/6.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

#### ELYZA tasksに対する応答の生成

ELYZA-tasks-100の`input`に対する応答を生成します。

ElyzaTasksHfGenerateManagerのインスタンス生成時に、'input'が含まれたjsonlファイルを`tasks_file_path`として指定します。

ファイルが指定されなかった場合は、自動的に`elyza/ELYZA-tasks-100`からダウンロードしたデータが使用されます。

In [8]:
from elyza_task.gen_manager_hf import ElyzaTasksHfGenerateManager

options = dict(
    # max_new_tokens=512,
    # max_new_tokens=256,
    max_new_tokens=128,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
    do_sample=False,
    repetition_penalty=1.2,
)

# elyza_jsonl = './data/test/elyza100_jp.jsonl'  # elyza-tasks-100を用いる場合は指定不要

gm = ElyzaTasksHfGenerateManager(
    hf_model=model,
    hf_tokenizer=tokenizer,
    template_type='llmjp3',
    # tasks_file_path=elyza_jsonl,
    **options,
)

gen_jsonl = '/content/elyza100_gen.jsonl'  # 生成するファイルパスを指定
gm(gen_jsonl_path=gen_jsonl)

README.md:   0%|          | 0.00/8.37k [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/115k [00:00<?, ?B/s]

Generating test split: 0 examples [00:00, ? examples/s]

0it [00:00, ?it/s]

ELYZA-tasks-100の`input`に対する応答出力が、`task_id`(0~99)と`output`キーに格納されたjsonlファイルが生成されます。

In [9]:
# ローカルに保存
from google.colab import files
files.download(gen_jsonl)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# # ラインタイム強制終了
# from google.colab import runtime
# runtime.unassign()